Informação importante

In [2]:
#Trecho retirado do kaggle no link do dataset escolhido: https://www.kaggle.com/datasets/mathchi/diabetes-data-set/data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings(action='ignore')

# Formato da base de dados:

##  Sobre:
 Todos os pacientes são mulheres com pelo menos 21 anos de idade e de ascendência indígena Pima.

## Análise exploratória dos dados (EDA):

In [3]:
import pandas as pd
#abrindo base dados
dataset = pd.read_csv("/kaggle/input/diabetes-data-set/diabetes.csv")
#imprimindo informações sobre a base dados
dataset.info()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/diabetes-data-set/diabetes.csv'

In [ ]:
dataset.shape

Em análise inicial vemos que o dataset tem o formato de tabela com 768 registros com 9 colunas sendo que, nenhuma do tipo texto.

In [ ]:
dataset.head()

O dataset é composto pelas colunas:
* Pregnancies: Número de vezes grávida
* Glucose: Concentração de glicose plasmática a 2 horas em um teste de tolerância oral à glicose
* BloodPressure: Pressão arterial diastólica (mm Hg)
* SkinThickness: Espessura da prega cutânea do tríceps (mm)
* Insulin: Insulina sérica de 2 horas (mu U/ml)
* BMI: Índice de massa corporal (peso em kg/(altura em m)^2)
* DiabetesPedigreeFunction: Função de pedigree do diabetes
* Age: Idade (anos)
* Outcome (variavel target): Variável de classe (0 ou 1)


Teremos como a variável "target" a coluna "Outcome".

In [ ]:
dataset.describe()

## Verificar a existencia de valores nulos

In [ ]:
dataset.isnull().sum()

Aparentemente o dataset não tem nenhum valor nulo

## Distribuição dos dados

In [ ]:
total = len(dataset)
diabetics = dataset[dataset['Outcome'] == 1].Outcome.count()
nonDiabetics = dataset[dataset['Outcome'] == 0].Outcome.count()

print(f"\nTotal de registros: {total}")
print(f"Diabeticos: {diabetics}")
print(f"Não Diabeticos: {nonDiabetics}")

### Visualização gráfica da distribuição:

In [ ]:
import matplotlib.pyplot as plt

colors = ['red', 'green']
labels = ['Diabéticos (1)', 'Não Diabéticos (0)']

plt.pie(dataset['Outcome'].value_counts(), labels=labels, colors=colors, autopct='%1.0f%%', textprops={'fontsize': 20})
plt.show()

## Analise das features

In [ ]:
import seaborn as sns

feature_names = [cname for cname in dataset.loc[:,:'Age'].columns]
plt.figure(figsize = (10,10))

i = 1
for feature in feature_names:
    plt.subplot(4,2,i)
    plt.title(feature)
    sns.histplot(x=feature, data = dataset, hue="Outcome")
    i = i + 1
plt.tight_layout()
plt.show()

### Correlação entre as Features

In [ ]:
correlation = dataset.corr()

fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(data=correlation, annot=True, linewidths=.5, ax=ax)

A feature com maior relação ao diagnostico de diabetes é a Glucose (0,47)

# Criação do Modelo

## Separação da base de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

x = dataset.drop('Outcome', axis=1)
y = dataset['Outcome']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=7) #20% para teste e

## Configurar o modelo

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

error = []

# Calculating error for K values between 1 and 10
for i in range(1, 10):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    pred_i = knn.predict(x_test)
    error.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(1, 10), error, color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize=10)
plt.title('Error Rate K Value')
plt.xlabel('K Value')
plt.ylabel('Mean Error')

In [ ]:
model = KNeighborsClassifier(n_neighbors=9)

In [ ]:
model.fit(x_train, y_train)

## Testando o modelo

In [ ]:
y_predicted = model.predict(x_test)

In [ ]:
dataset.groupby('Outcome').count()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

confusion_matrix = confusion_matrix(y_true = y_test,
                                   y_pred = y_predicted,
                                   labels= [1, 0])

figure = plt.figure(figsize=(15, 5))
disp = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix,
                              display_labels= [1, 0])

disp.plot(values_format='d')

True Positive (TP): 31

True Negative (TN): 84

False Positive (FP): 16

False Negative (FN): 23

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predicted))

## Testando KNN com validação Cruzada:

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kfold  = KFold(n_splits=5, shuffle=True)
result = cross_val_score(model, x, y, cv = kfold)

print("K-Fold (R^2) Scores: {0}".format(result))
print("Mean R^2 for Cross-Validation K-Fold: {0}".format(result.mean()))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score

# Buscar os melhores parâmetros
# Vamos usar uma técnica chamada Gridsearch que é basicamente a busca por força bruta
# Iremos utilizar a técnica de validação cruzada de 5 folds (divisões) em cima do conjunto de treinamento
# Como métrica de melhor desemepnho usaremos a acurácia, isto é, estamos buscando os hiperparâmetros que maximizam a acurácia

# Parâmetros testados
param_grid = {
            'n_neighbors':[3,9], #total de vizinhos
            'weights': ['uniform', 'distance'], #Função de peso usada na previsão
            'metric':['cosine', 'euclidean', 'manhattan'] #métrica para cálculo da distância
             }

# Métrica de desempenho = melhor acurácia
gs_metric = make_scorer(accuracy_score, greater_is_better=True)

grid = GridSearchCV(
    KNeighborsClassifier(),
    param_grid=param_grid,
    scoring=gs_metric,
    cv=5,
    n_jobs=4,
    verbose=3
) #cv = gerador de validação cruzada / n_jobs = determina a quantidade de jobs processados

grid.fit(x_train, y_train) #Selecionando os melhores hiperparametros com base nos dados de treino
knn_params = grid.best_params_
print('KNN', knn_params)

In [ ]:
grid.cv_results_

## Testando vários tipos de algoritmos de classificação:

In [ ]:
def AplicaValidacaoCruzada(x_axis, y_axis):
  # Linear Models.
  from sklearn.neighbors import KNeighborsClassifier  # k-vizinhos mais próximos (KNN)
  from sklearn.ensemble import RandomForestClassifier # RandomForest
  from sklearn.svm import SVC                         # Maquina de Vetor Suporte SVM

  # Cross-Validation models.
  from sklearn.model_selection import cross_val_score
  from sklearn.model_selection import KFold

  # Configuração de KFold.
  kfold  = KFold(n_splits=10, shuffle=True)

  # Axis
  x = x_axis
  y = y_axis

  # Criando os modelos

  # KNN
  knn = KNeighborsClassifier(n_neighbors=8, metric= 'euclidean', weights='distance')
  knn.fit(x_train, y_train)

  # SVM
  svm = SVC()
  svm.fit(x_train, y_train)

  # RandomForest
  rf = RandomForestClassifier(random_state=7)
  rf.fit(x_train, y_train)

  # Applyes KFold to models.
  knn_result = cross_val_score(knn, x, y, cv = kfold)
  svm_result = cross_val_score(svm, x, y, cv = kfold)
  rf_result = cross_val_score(rf, x, y, cv = kfold)

  # Creates a dictionary to store Linear Models.
  dic_models = {
    "KNN": knn_result.mean(),
    "SVM": svm_result.mean(),
    "RF": rf_result.mean()
  }
  # Select the best model.
  melhorModelo = max(dic_models, key=dic_models.get)

  print("KNN (R^2): {0}\nSVM (R^2): {1}\nRandom Forest (R^2): {2}".format(knn_result.mean(), svm_result.mean(), rf_result.mean()))
  print("O melhor modelo é : {0} com o valor: {1}".format(melhorModelo, dic_models[melhorModelo]))

In [ ]:
AplicaValidacaoCruzada(x, y)

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

y_prob = model.predict_proba(x_test)[:,1]
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate,true_positive_rate, color='red',label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],linestyle='--')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')